In [23]:
"""
    Fernando Martins Pimenta
    
    Get the bounding box of each feature in a shapefile 
    to generate polygons based on their bounding boxes.
"""

import shapefile
import geopandas as gpd
import pandas as pd
from shapely.geometry import Polygon
from osgeo import osr
import os

cwd = os.getcwd()

sf = shapefile.Reader('areacontribuicaon5.shp')
shapes = sf.shapes() 

rec = sf.records()

data = pd.DataFrame(index=range(len(rec)), columns=['ottocod', 'xmin', 'ymin','xmax', 'ymax'])

for i in range(len(rec)):
    data['ottocod'][i] = rec[i][0]
    data['xmin'][i] = shapes[i].bbox[0]
    data['ymin'][i] = shapes[i].bbox[1]
    data['xmax'][i] = shapes[i].bbox[2]
    data['ymax'][i] = shapes[i].bbox[3]
    
data.to_csv('basin_bbox.csv', sep = ',')

target_epsg = 4326

# Create a polygon shapefile
w = shapefile.Writer(cwd+'/XXX.shp')
w.shapeType = 5

w.field('ottocod','N')

pol = [[
    [data['xmin'][0],data['ymin'][0]],
    [data['xmin'][0],data['ymax'][0]],
    [data['xmax'][0],data['ymax'][0]],
    [data['xmax'][0],data['ymin'][0]],
    [data['xmin'][0],data['ymin'][0]]
]]

w.poly(pol)

w.record(data['ottocod'][0])

w.close()

dst_file = cwd+'/XXX.shp'

## create ESRI prj-file
sr = osr.SpatialReference()
sr.ImportFromEPSG(target_epsg)
sr.MorphToESRI()
with open(dst_file.replace('.shp', '.prj'), 'w') as prj:
    prj.write(sr.ExportToWkt())

In [21]:
help(w.field)

Help on method field in module shapefile:

field(name, fieldType='C', size='50', decimal=0) method of shapefile.Writer instance
    Adds a dbf field descriptor to the shapefile.

